# Utah Real Estate Market Analysis: Sales Trends and Predictive Modeling

## Overview

The Utah Real Estate Sales Analysis project aims to leverage a dataset containing detailed real estate listings from Utah to explore market trends, pricing dynamics, and key property characteristics influencing sales prices. The dataset provides comprehensive information about 4,440 unique real estate listings, including attributes such as property type, size, listing price, bedrooms, bathrooms, lot size, garage spaces, and the date of last sale. By analyzing this data, we seek to gain insights into the Utah real estate market and develop predictive models to estimate property prices based on various features.

# Exploring the Data

## Data Understanding
As previously mentioned, the dataset used in this analysis is sourced from Utah real estate listings and provides detailed information for 4,440 properties, which were built as early as 1860 to 2026 for future listings. The dataset allows for a comprehensive analysis of factors influencing property values and trends within Utah’s real estate market.

## 1.1 Load the Datasets
During this process, we will load and prepare our data for an exploratory data analysis by gaining key insights from our data by taking a general look (.head), checking for missing values (.info), and a brief summary of statistics (.describe).

In [1]:
#Import and load the data with panda
import pandas as pd
path = '../data/real_estate_utah.csv'
real_estate_data = pd.read_csv(path)

In [2]:
real_estate_data.head()

,type,text,year_built,beds,baths,baths_full,baths_half,garage,lot_sqft,sqft,stories,lastSoldOn,listPrice,status
0,single_family,"Escape to tranquility with this off-grid, unfi...",2020.0,1.0,1.0,1.0,1.0,2.0,71438.0,696.0,2.0,2018-05-31,90000.0,for_sale
1,single_family,Beautiful home in the desirable Oak Hills and ...,1968.0,4.0,3.0,2.0,1.0,2.0,56628.0,3700.0,2.0,2018-05-31,799000.0,for_sale
2,single_family,"Welcome to your new home, nestled in the heart...",1985.0,4.0,3.0,3.0,1.0,1.0,10019.0,3528.0,2.0,2018-05-31,389900.0,for_sale
3,single_family,Investment Opportunity. House needs some work ...,1936.0,4.0,2.0,2.0,1.0,2.0,12632.0,2097.0,2.0,2018-04-16,300000.0,for_sale
4,land,Deer Springs Ranch is an 8000 Ac Ranch in an H...,2003.0,4.0,0.0,2.0,1.0,2.0,872071.0,2400.0,2.0,2018-05-31,70000.0,for_sale


In [3]:
real_estate_data.tail()

,type,text,year_built,beds,baths,baths_full,baths_half,garage,lot_sqft,sqft,stories,lastSoldOn,listPrice,status
4435,single_family,"New, move-in ready 3-bed 2-bath home by Zions ...",2024.0,3.0,2.0,2.0,1.0,2.0,5227.0,1542.0,1.0,2018-05-31,465000.0,for_sale
4436,single_family,Located in the new elite community of The Mant...,2024.0,3.0,2.0,2.0,1.0,2.0,7405.0,4230.0,3.0,2018-05-31,570465.0,for_sale
4437,single_family,"Beautiful Maeser location, 3 bedroom, 2 bath h...",1931.0,3.0,2.0,1.0,1.0,2.0,18295.0,1903.0,2.0,2008-09-30,475000.0,for_sale
4438,single_family,NEW PRICE! Best Value on the Market! Quick Mov...,2021.0,6.0,3.0,3.0,1.0,2.0,8712.0,4000.0,2.0,2021-09-08,629900.0,for_sale
4439,single_family,Welcome to this beautiful custom built home th...,2015.0,6.0,4.0,3.0,1.0,3.0,23958.0,4382.0,3.0,2016-02-26,1200000.0,for_sale


In [4]:
real_estate_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4440 entries, 0 to 4439
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   type        4440 non-null   object 
 1   text        4440 non-null   object 
 2   year_built  4440 non-null   float64
 3   beds        4440 non-null   float64
 4   baths       4440 non-null   float64
 5   baths_full  4440 non-null   float64
 6   baths_half  4440 non-null   float64
 7   garage      4440 non-null   float64
 8   lot_sqft    4440 non-null   float64
 9   sqft        4440 non-null   float64
 10  stories     4440 non-null   float64
 11  lastSoldOn  4440 non-null   object 
 12  listPrice   4440 non-null   float64
 13  status      4440 non-null   object 
dtypes: float64(10), object(4)
memory usage: 485.8+ KB


In [7]:
real_estate_data.describe()

,year_built,beds,baths,baths_full,baths_half,garage,lot_sqft,sqft,stories,listPrice
count,4440.000000,4440.000000,4440.000000,4440.000000,4440.000000,4440.000000,4.440000e+03,4440.000000,4440.000000,4.440000e+03
mean,1997.942793,3.894595,2.454730,2.238964,1.025000,2.334234,5.525239e+05,2712.318243,1.999775,7.966044e+05
std,23.609067,1.265647,1.789238,1.165071,0.194672,1.024810,1.134471e+07,1553.676432,0.629134,1.731703e+06
min,1860.000000,1.000000,0.000000,1.000000,1.000000,0.000000,4.360000e+02,0.000000,1.000000,0.000000e+00
25%,1997.000000,3.000000,2.000000,2.000000,1.000000,2.000000,9.583000e+03,1842.000000,2.000000,3.538050e+05
50%,2003.000000,4.000000,3.000000,2.000000,1.000000,2.000000,1.393900e+04,2400.000000,2.000000,5.289950e+05
75%,2007.000000,4.000000,3.000000,3.000000,1.000000,2.000000,2.439400e+04,3132.000000,2.000000,7.549000e+05
max,2026.000000,19.000000,45.000000,45.000000,6.000000,20.000000,6.009538e+08,20905.000000,4.000000,4.800000e+07


In [8]:
real_estate_data.shape

(4440, 14)

- There are 14 columns/features to be evaluated on
- 4,400 records and no missing values

## 1.2 Understanding the Structure

### Real Estate Data Key Features
Column Descriptors (Pulled from Kaggle)
- type: Type of property (e.g., single_family, land)
- text: Description of the property
- year_built: Year the property was built
- beds: Number of bedrooms
- baths: Total number of bathrooms
- baths_full: Number of full bathrooms
- baths_half: Number of half bathrooms
- garage: Number of garage spaces
- lot_sqft: Lot size in square feet
- sqft: Property size in square feet
- stories: Number of stories
- lastSoldOn: Date the property was last sold
- listPrice: Listing price of the property
- status: Current status of the property (e.g., for_sale)

Key Data Insights based on Summary Stats from .describe:
- potential outliers in the sqft, lot_sqft, and listPrice columns
- Data types are generally correct except for lastSoldOn, type, and status

# Data Cleaning

Initial exploration reveals possible outliers in variables like lot_sqft, sqft, and listPrice, with some unusually high values that may skew analysis. Handling these outliers will involve filtering or capping based on the overall distribution. The lastSoldOn column, representing dates, will be transformed into a more analytical format (e.g., extracting year or month). For categorical variables like type and status, label encoding or one-hot encoding will be applied based on the context of each analysis. After completing these transformations, the cleaned dataset will be exported in a standardized format for seamless integration with other analysis notebooks.

## 2.1 Handle Missing Values
Our .info does not show any current missing null values, but for a sanity check let's take a look for null values again in case we missed anything using isnull() and sum() to identify columns with missing data.


In [9]:
real_estate_data.isnull().sum()

type          0
text          0
year_built    0
beds          0
baths         0
baths_full    0
baths_half    0
garage        0
lot_sqft      0
sqft          0
stories       0
lastSoldOn    0
listPrice     0
status        0
dtype: int64

## 2.2 Data Type Identification
Check and ensure that the data types (integer, float, object, datetime) are correct using .dtypes.
Convert columns to appropriate data types (e.g., categorical, datetime) using pd.to_datetime() or .astype()

In [11]:
real_estate_data.dtypes

type           object
text           object
year_built    float64
beds          float64
baths         float64
baths_full    float64
baths_half    float64
garage        float64
lot_sqft      float64
sqft          float64
stories       float64
lastSoldOn     object
listPrice     float64
status         object
dtype: object

EDA Treatment
- `lastSoldOn` Convert to datetime, extract year or recency
- `year_built` Convert to age, analyze distribution
- `sqft` Analyze distribution, check skew, possibly log transform
- `lot_sqft`  Check for skew, use log transform if needed



### Analyzing Categorical Columns

In [18]:
#Analyzing the values in the type columns 
real_estate_data['type'].value_counts()

type
single_family                  2883
land                            801
townhomes                       344
mobile                          206
condos                          156
townhouse                        14
other                            12
farm                              9
condo_townhome_rowhome_coop       8
condo_townhome                    6
condo                             1
Name: count, dtype: int64

Grouping all condo-related types and townhouse/townhomes into a single category will standardize this feature, reducing redundancy and improving consistency across analyses. This simplification minimizes noise, making it easier to compare property types in EDA and inferential statistics and helps ML models generalize better by focusing on meaningful distinctions between broader categories.

In [19]:
# List of condo-related types to merge
condo_types = ['condos', 'condo', 'condo_townhome_rowhome_coop', 'condo_townhome',]

# Replace condo-related types with a single category 'condo'
real_estate_data['type'] = real_estate_data['type'].replace(condo_types, 'condo')

#list of townhouse types to merge
townhouse_types = ['townhouse', 'townhomes']

#replace townhouse related types with a single category 'townhouse'
real_estate_data['type'] = real_estate_data['type'].replace(townhouse_types, 'townhouse')

# Check to ensure the types are updated
print(real_estate_data['type'].value_counts())



type
single_family    2883
land              801
townhouse         358
mobile            206
condo             171
other              12
farm                9
Name: count, dtype: int64


### Dropping the text column
We can drop the text column because we won't be using it. However if we wanted to extend this project for further exploration we could keep it for NLP analysis

In [24]:
real_estate_data.drop(columns=['text'], axis=1, inplace=True)

## 2.3 Date Standardization for lastSoldOn Column
The lastSoldOn column, representing dates, will be converted to DateTime format, allowing extraction of specific components like year and month for deeper time-based analysis.

In [17]:
# Convert 'lastSoldOn' to datetime
real_estate_data['lastSoldOn'] = pd.to_datetime(real_estate_data['lastSoldOn'], errors='coerce')

# Extract year and month for analysis
real_estate_data['sold_year'] = real_estate_data['lastSoldOn'].dt.year
real_estate_data['sold_month'] = real_estate_data['lastSoldOn'].dt.month


### 2.35 Year Standardization 
For EDA, we are going to convert the year_built to the property age. Keeping the year_built column while creating a new property_age column can be beneficial. Adding property_age allows for a more intuitive interpretation of a property's age, making it easier to analyze trends and relationships with the listing price, while still preserving the original year_built data for any additional historical insights. 

In [21]:
real_estate_data['year_built'] = real_estate_data['year_built'].astype('int')
real_estate_data['age'] = pd.to_datetime('today').year - real_estate_data['year_built']
real_estate_data['age'].value_counts()

age
21     1690
0       229
2        98
1        97
3        90
       ... 
139       1
133       1
125       1
143       1
141       1
Name: count, Length: 140, dtype: int64

## 2.4 Detect and Removing Outliers
Potential outliers with abnormally high values that might affect the study were found in the lot_sqft, sqft, and listPrice columns during preliminary investigation. Depending on the distribution of the data, we will identify these outliers and implement filtering or capping using numpy

In [26]:
# import numpy as np

# # Set thresholds based on percentiles to cap or filter out extreme values
# cap_thresholds = {
#     'lot_sqft': np.percentile(real_estate_data['lot_sqft'], 99),
#     'sqft': np.percentile(real_estate_data['sqft'], 99),
#     'listPrice': np.percentile(real_estate_data['listPrice'], 99)
# }

# # Capping outliers at the 99th percentile
# real_estate_data['lot_sqft'] = np.where(real_estate_data['lot_sqft'] > cap_thresholds['lot_sqft'], cap_thresholds['lot_sqft'], real_estate_data['lot_sqft'])
# real_estate_data['sqft'] = np.where(real_estate_data['sqft'] > cap_thresholds['sqft'], cap_thresholds['sqft'], real_estate_data['sqft'])
# real_estate_data['listPrice'] = np.where(real_estate_data['listPrice'] > cap_thresholds['listPrice'], cap_thresholds['listPrice'], real_estate_data['listPrice'])


In [27]:
# # Check summary statistics again after capping to see if extreme values remain
# real_estate_data[['lot_sqft', 'sqft', 'listPrice']].describe(percentiles=[0.25, 0.5, 0.75, 0.9, 0.95, 0.99])


After handling the outliers, it appears that extreme values in "lot_sqft," "sqft," and "listPrice" have been capped. The maximum values have been significantly reduced, especially in the "lot_sqft" and "listPrice" columns, where the original maximums were in the range of tens of millions, but the capped values are much lower, under 7 million for "listPrice" and around 6 million for "lot_sqft." The 99th percentile also shows more reasonable values after capping.

## 2.45 Revision Detecting & Removing Outliers using IQR (Produces better Prediction Models)
The accuracy of our prediction models may be limited due to the significant number of outliers still present in the data, impacting model performance. To improve results, we will apply the IQR method, similar to Mohammadreza Mohammadi’s approach, to remove outliers instead of capping them, which will be reflected in the ML notebook

To view his approach: https://www.kaggle.com/code/jigsaw13/utah-analysis-and-prediction/notebook#4.1-Outliers-%3Ca-id-4.%3E%3C/a%3E

In [25]:
def remove_outliers_iqr(real_estate_data, columns):
    """
    Remove outliers from the specified columns in the DataFrame using the IQR technique.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    columns (list): A list of column names to check for outliers.

    Returns:
    pd.DataFrame: A DataFrame with outliers removed.
    """
    for column in columns:
        Q1 = real_estate_data[column].quantile(0.25)
        Q3 = real_estate_data[column].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - (1.5 * IQR)
        upper_bound = Q3 + (1.5 * IQR)

        real_estate_data = real_estate_data[(real_estate_data[column] >= lower_bound) & (real_estate_data[column] <= upper_bound)]

    return real_estate_data

In [26]:
#Check for new columns
real_estate_data.head()

,type,year_built,beds,baths,baths_full,baths_half,garage,lot_sqft,sqft,stories,lastSoldOn,listPrice,status,sold_year,sold_month,age
0,single_family,2020,1.0,1.0,1.0,1.0,2.0,71438.0,696.0,2.0,2018-05-31,90000.0,for_sale,2018,5,4
1,single_family,1968,4.0,3.0,2.0,1.0,2.0,56628.0,3700.0,2.0,2018-05-31,799000.0,for_sale,2018,5,56
2,single_family,1985,4.0,3.0,3.0,1.0,1.0,10019.0,3528.0,2.0,2018-05-31,389900.0,for_sale,2018,5,39
3,single_family,1936,4.0,2.0,2.0,1.0,2.0,12632.0,2097.0,2.0,2018-04-16,300000.0,for_sale,2018,4,88
4,land,2003,4.0,0.0,2.0,1.0,2.0,872071.0,2400.0,2.0,2018-05-31,70000.0,for_sale,2018,5,21


## 2.4 Encoding Categorical Variables
Categorical variables like type and status will be encoded. For variables with ordinal significance, label encoding will be used, while one-hot encoding will apply for nominal categories.

### Handling the Type Column:
We are going to create a new column that categorizes the 'type' column into individual property types compared to "Other". For example, if the property type is "Single Family", it will remain as "Single Family", but if it's any other type, it will be classified as "Other". We are doing this to keep the original type column to answer questions for EDA, but drop it once we get to constructing the machine learning model. Perform one-hot encoding on the "type" column to create separate columns for each individual type. Concatenate both the new comparison column and the one-hot encoded columns.

In [31]:
# Create a new 'type_comparison' column where each 'type' is compared to 'Other'
real_estate_data['type_comparison'] = real_estate_data['type'].apply(lambda x: x if x == 'Single Family' else 'Other')

# One-hot encoding for the 'type' column (original categorical column)
type_dummies = pd.get_dummies(real_estate_data['type'], prefix='type')

# Concatenate the new column and the one-hot encoded columns with the original DataFrame
real_estate_data = pd.concat([real_estate_data, type_dummies], axis=1)


### Handling the Status Column

For the label encoding the status: 0 represents if the property is for sale and 1 represents if the property is ready to build

In [27]:
from sklearn.preprocessing import LabelEncoder

# Label Encoding for ordinal categories (status)
label_encoder = LabelEncoder()
real_estate_data['status_encoded'] = label_encoder.fit_transform(real_estate_data['status'])
real_estate_data['status_encoded'].value_counts()

status_encoded
0    4185
1     255
Name: count, dtype: int64

## 2.5 Drop Duplicate values
We can identify the duplicate columns by their naming pattern (e.g., the .1, .2, .3 suffix) and drop them, keeping only the first occurrence of each encoded column (e.g., type_condo, type_condo_townhome, etc.).

In [28]:
# Get a list of columns that match the pattern for duplicate encoded columns (e.g., columns with '.1', '.2', '.3' suffixes)
duplicate_columns = [col for col in real_estate_data.columns if '.' in col and col.split('.')[0] in real_estate_data.columns]

# Drop the duplicates, keeping the first occurrence
real_estate_data = real_estate_data.drop(columns=duplicate_columns)

# Check the updated columns in the dataset
real_estate_data.columns


Index(['type', 'year_built', 'beds', 'baths', 'baths_full', 'baths_half',
       'garage', 'lot_sqft', 'sqft', 'stories', 'lastSoldOn', 'listPrice',
       'status', 'sold_year', 'sold_month', 'age', 'status_encoded'],
      dtype='object')

## 2.5 Exporting the Cleaned Dataset
After completing these transformations, the cleaned dataset will be exported in a standardized format to integrate seamlessly with other analysis notebooks.

In [29]:
# Exporting the cleaned data to a CSV file for further analysis
real_estate_data.to_csv('cleaned_real_estate_utah.csv', index=False)
